In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from fitpass import fitpass_optimization
from fitpass import get_studios
import folium

In [5]:
def print_in_map(studios_1,times,colors,size):
    for i, row in studios_1[studios_1['gym_times'] == times].iterrows():
        label = row['gym_name']+ ": times -"+ str(row['gym_times'])
        folium.Marker(
            location=[row.latitude, row.longitude],
            tooltip=label,
            icon=folium.Icon(
                color=colors,
                icon_color="black",
                weight=1,
                fill_opacity="black"
                )
            ).add_to(m)

In [27]:
user1 = {
    # user info
    "name": "Javier Montiel",
    # location
    "location": {
        "longitude": -99.17734895911873,
        "latitude": 19.394777855494667,
    },
    # user loss distance to studios
    "distance_sensitivity": 'medium', # low, medium, high
    # user preferences
    "preferences": {
        "love_activities": ["cycling", "gym", "dance", "sports"],
        "hate_activities": ["pilates", "hiit", "yoga", "box"],
    },
    # constraints
    "is_pro": 1,
    "max_allowed_classes_per_class": 4,
    "num_classes_per_month": 26,
}

In [28]:
# example weights
WEIGHTS = {
    'distance': 0.65,
    'preference': 0.1,
    'activity': 0.25
}


In [29]:
df_sol, fmin = fitpass_optimization(user1, WEIGHTS)
df_sol

,gym_name,gym_times,activity,distance,preference_score,pro_status,geometry
0,Fitness Pilates Napoles WTC,4,[pilates],0.346785,0.266667,0,POINT (2795969.507 824768.108)
1,Flow Pilates Nápoles,4,[pilates],0.353151,0.266667,0,POINT (2795792.133 824481.657)
2,Kodokan México,2,[mma],0.543472,0.466667,0,POINT (2796162.513 824861.778)
3,MetabolicWorkout Barre,1,"[barre, box, crossfit, dance, functional, gym,...",1.889353,0.600000,0,POINT (2795837.877 826669.126)
4,Olympica Sports Performance,1,"[crossfit, cycling]",0.762088,0.600000,0,POINT (2796302.809 824446.459)
5,Pandora Beauty Lab,4,[wellness],0.610142,0.600000,1,POINT (2796105.495 825166.130)
6,Ser Yoga,4,"[wellness, yoga]",0.423314,0.266667,0,POINT (2795686.570 824373.373)
7,Sports World Manacar,1,"[gym, pool]",2.926827,1.000000,1,POINT (2795296.037 821883.523)
8,Sports World Patriotismo,4,[gym],0.402431,1.000000,1,POINT (2795250.558 824943.072)
9,Vie Spa del Valle,1,"[ems, wellness]",0.691692,0.600000,1,POINT (2796314.764 824817.530)


In [30]:
studios = get_studios()
results = df_sol
user = user1
zoom = 13


studios_1 = studios.merge(results[["gym_name", "gym_times"]], on = 'gym_name', how = 'left')
studios_1 = studios_1.fillna(0)
user1df = pd.DataFrame({'gym_name' : user.get("name"), 'latitude' : [user.get("location").get("latitude")] , 'longitude' : [user.get("location").get("longitude")], 'gym_times' : "No_aplica"})
studios_1 = pd.concat([studios_1,user1df])
m = folium.Map(location=[user.get("location").get("latitude"), user.get("location").get("longitude")], tiles='cartodbpositron', zoom_start=zoom)



#print_in_map(studios_1, 0,"gray",0.5)
print_in_map(studios_1,1,"beige",2)
print_in_map(studios_1, 2, "lightgreen",2)
print_in_map(studios_1,3,"green",2)
print_in_map(studios_1,4,"darkgreen",2)
print_in_map(studios_1,"No_aplica","red",3)

m